# USR Phase Space Analysis: Fine Tuning Heatmap

## Objective
Explore the 'Phase Space' of initial conditions $(\phi_i, y_i)$ for a fixed non-minimal coupling $\xi$.
We aim to identify regions (islands) in initial conditions that lead to a significant **Ultra-Slow-Roll (USR)** phase.

**Correction:** This version truncates the trajectory analysis at the end of inflation ($\epsilon_H \ge 1$) to avoid numerical noise from post-inflationary oscillations being misidentified as USR.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add root path for imports
root_dir = os.path.abspath('..')
if root_dir not in sys.path:
    sys.path.insert(0, root_dir)

from inflation_models import HiggsModel
from inf_dyn_background import run_background_simulation, get_derived_quantities

# Plot Settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# ==========================================
# 1. Configuration & Ranges
# ==========================================

# Model Constants
XI_FIXED = 100.0
LAM_FIXED = 0.1

# Grid Settings (Resolution increased for KD scan)
N_PHI = 20
N_Y = 20

# Ranges
phi_min, phi_max = 5.5, 6.5
# Log velocity range: 0.1 to 100 (Kinetic Dominance)
    # magnitudes from 10^-1 to 10^2
    y_min_log, y_max_log = -1.0, 2.0 

phi_grid = np.linspace(phi_min, phi_max, N_PHI)
# We assume negative velocity (rolling down)
# logspace(-5, 1) gives positive magnitudes 0.00001 to 10
y_grid_magnitudes = np.logspace(y_min_log, y_max_log, N_Y)
y_grid = -y_grid_magnitudes 

print(f"Scanning grid: {N_PHI}x{N_Y} = {N_PHI*N_Y} simulations.")
print(f"Phi range: [{phi_min}, {phi_max}]")
print(f"Velocity range: [-{10**y_max_log}, -{10**y_min_log}]")

In [ ]:
# ==========================================
# 2. Trajectory Analysis Logic (Corrected)
# ==========================================

def analyze_trajectory(model, T_max):
    """
    Runs sim and returns: (usr_duration, min_eta, N_total)
    Truncates analysis at end of inflation to verify true physics.
    """
    # Standard Run
    try:
        # Run background simulation
        sol = run_background_simulation(model, np.linspace(0, T_max, 2000))
    except Exception:
        return 0.0, 0.0, 0.0
    
    # Check for valid output
    if np.any(np.isnan(sol)):
        return 0.0, 0.0, 0.0
        
    data = get_derived_quantities(sol, model)
    
    N = data['N']
    eta = data['etaH']
    eps = data['epsH']

    # 1. Detect End of Inflation & Truncate
    ended = eps >= 1
    if np.any(ended):
        idx_end = np.argmax(ended)
        N_total = N[idx_end]
        
        # CRITICAL FIX: Only analyze the inflationary phase
        # Slice arrays up to end of inflation
        eta_inf = eta[:idx_end+1]
        N_inf = N[:idx_end+1]
    else:
        # Inflation didn't end within T_max
        N_total = N[-1]
        eta_inf = eta
        N_inf = N

    # 2. USR Detection (on truncated data)
    # USR is defined as eta < -2 (approx -6 usually)
    is_usr = (eta_inf < -2.0)
    
    if np.any(is_usr):
        # Integrate dN over USR regions
        dN = np.diff(N_inf, prepend=N_inf[0])
        duration = np.sum(dN[is_usr])
        min_eta_val = np.min(eta_inf)
    else:
        duration = 0.0
        min_eta_val = np.min(eta_inf) if len(eta_inf) > 0 else 0.0
        
    return duration, min_eta_val, N_total

In [ ]:
# ==========================================
# 3. Execution (Serial)
# ==========================================

grid_duration = np.zeros((N_Y, N_PHI))
grid_min_eta = np.zeros((N_Y, N_PHI))
grid_N_total = np.zeros((N_Y, N_PHI))

# Simulation time max
T_MAX_SCALED = max(200, 2.0 * XI_FIXED)

print(f"Starting Scan (Series)... T_max = {T_MAX_SCALED}")

for i, y_val in enumerate(y_grid):
    # Progress bar row-by-row
    print(f"Processing row {i+1}/{N_Y} (y = {y_val:.2e})...", end='\r')
    
    for j, phi_val in enumerate(phi_grid):
        model = HiggsModel(lam=LAM_FIXED, xi=XI_FIXED)
        model.phi0 = phi_val
        model.yi = y_val
        dur, meta, ntot = analyze_trajectory(model, T_MAX_SCALED)
        
        grid_duration[i, j] = dur
        grid_min_eta[i, j] = meta
        grid_N_total[i, j] = ntot

print("\nScan Complete.")

In [ ]:
# ==========================================
# 4. Visualization
# ==========================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# We plot against magnitude of y for log scale
X, Y = np.meshgrid(phi_grid, np.abs(y_grid))

# Plot 1: USR Duration
pcm1 = ax1.pcolormesh(X, Y, grid_duration, shading='auto', cmap='magma')
ax1.set_yscale('log')
ax1.set_xlabel(r'Initial Field $\phi_i [M_P]$')
ax1.set_ylabel(r'Initial Speed $|y_i|$')
ax1.set_title(rf'USR Phase Duration (e-folds)\n$\xi={XI_FIXED}, \eta < -2$')
fig.colorbar(pcm1, ax=ax1, label='Duration [N]')

# Plot 2: Total N
pcm2 = ax2.pcolormesh(X, Y, grid_N_total, shading='auto', cmap='viridis')
ax2.set_yscale('log')
ax2.set_xlabel(r'Initial Field $\phi_i [M_P]$')
ax2.set_ylabel(r'Initial Speed $|y_i|$')
ax2.set_title(rf'Total Inflation Duration $N_{{tot}}$')
fig.colorbar(pcm2, ax=ax2, label='Total e-folds')

plt.tight_layout()
plt.show()